In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing
from hmmlearn import hmm
from sklearn import metrics

In [2]:
# importing data from 2 different .csv files. Concatenating data from 2 files

data = pd.read_csv(r'application_train.csv')
data_bureau = pd.read_csv(r'bureau.csv')

labels = ['DAYS_CREDIT','CREDIT_DAY_OVERDUE','DAYS_CREDIT_ENDDATE','DAYS_ENDDATE_FACT','DAYS_CREDIT_UPDATE']

feature_importance =['TARGET','DAYS_BIRTH','DAYS_REGISTRATION','DAYS_ID_PUBLISH','DAYS_EMPLOYED','DAYS_LAST_PHONE_CHANGE',
                'AMT_ANNUITY','AMT_INCOME_TOTAL','REGION_POPULATION_RELATIVE','AMT_CREDIT','HOUR_APPR_PROCESS_START',
                'AMT_GOODS_PRICE','ORGANIZATION_TYPE','OWN_CAR_AGE','WEEKDAY_APPR_PROCESS_START','OCCUPATION_TYPE',
                'NAME_FAMILY_STATUS','OBS_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','CNT_FAM_MEMBERS',
                'NAME_TYPE_SUITE','NAME_HOUSING_TYPE','NAME_EDUCATION_TYPE','NAME_INCOME_TYPE','CNT_CHILDREN',
                'DEF_30_CNT_SOCIAL_CIRCLE','FLAG_OWN_REALTY','DEF_60_CNT_SOCIAL_CIRCLE','FLAG_WORK_PHONE',
                'FLAG_PHONE','REGION_RATING_CLIENT_W_CITY','REG_CITY_NOT_LIVE_CITY','REGION_RATING_CLIENT','CODE_GENDER',
                'REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY','FLAG_EMAIL','FLAG_DOCUMENT_3','FLAG_OWN_CAR',
                'FLAG_DOCUMENT_8','LIVE_REGION_NOT_WORK_REGION','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','NAME_CONTRACT_TYPE',
                'REG_REGION_NOT_WORK_REGION','REG_REGION_NOT_LIVE_REGION','FLAG_DOCUMENT_18','FLAG_DOCUMENT_16'] 

df = pd.DataFrame(data,columns = feature_importance)
bf= pd.DataFrame(data_bureau,columns = labels)
bf = bf[0:len(df)]
df = pd.concat([df,bf],axis=1)
df

,TARGET,DAYS_BIRTH,DAYS_REGISTRATION,DAYS_ID_PUBLISH,DAYS_EMPLOYED,DAYS_LAST_PHONE_CHANGE,AMT_ANNUITY,AMT_INCOME_TOTAL,REGION_POPULATION_RELATIVE,AMT_CREDIT,...,NAME_CONTRACT_TYPE,REG_REGION_NOT_WORK_REGION,REG_REGION_NOT_LIVE_REGION,FLAG_DOCUMENT_18,FLAG_DOCUMENT_16,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,DAYS_CREDIT_UPDATE
0,1,-9461,-3648.0,-2120,-637,-1134.0,24700.5,202500.0,0.018801,406597.5,...,Cash loans,0,0,0,0,-497,0,-153.0,-153.0,-131
1,0,-16765,-1186.0,-291,-1188,-828.0,35698.5,270000.0,0.003541,1293502.5,...,Cash loans,0,0,0,0,-208,0,1075.0,NaN,-20
2,0,-19046,-4260.0,-2531,-225,-815.0,6750.0,67500.0,0.010032,135000.0,...,Revolving loans,0,0,0,0,-203,0,528.0,NaN,-16
3,0,-19005,-9833.0,-2437,-3039,-617.0,29686.5,135000.0,0.008019,312682.5,...,Cash loans,0,0,0,0,-203,0,NaN,NaN,-16
4,0,-19932,-4311.0,-3458,-3038,-1106.0,21865.5,121500.0,0.028663,513000.0,...,Cash loans,0,0,0,0,-629,0,1197.0,NaN,-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,-9327,-8456.0,-1982,-236,-273.0,27558.0,157500.0,0.032561,254700.0,...,Cash loans,0,0,0,0,-147,0,1679.0,NaN,-49
307507,0,-20775,-4388.0,-4090,365243,0.0,12001.5,72000.0,0.025164,269550.0,...,Cash loans,0,0,0,0,-1378,0,-1227.0,-1226.0,-1082
307508,0,-14966,-6737.0,-5150,-7921,-1909.0,29979.0,153000.0,0.005002,677664.0,...,Cash loans,0,0,0,0,-2178,0,-1966.0,-1966.0,-1966
307509,1,-11961,-2562.0,-931,-4786,-322.0,20205.0,171000.0,0.005313,370107.0,...,Cash loans,0,0,0,0,-1189,0,638.0,-833.0,-833


In [3]:
# Separating X and Y variables and transforming from string variables to numerical values

column_names = feature_importance+labels
X = df[column_names]
X = df.drop(['TARGET'], axis=1)
Y = df.TARGET

le = preprocessing.LabelEncoder()
le.fit(X['NAME_CONTRACT_TYPE'])
X['NAME_CONTRACT_TYPE'] = le.transform(X['NAME_CONTRACT_TYPE'])

le.fit(X['CODE_GENDER'])
X['CODE_GENDER'] = le.transform(X['CODE_GENDER'])

le.fit(X['FLAG_OWN_CAR'])
X['FLAG_OWN_CAR'] = le.transform(X['FLAG_OWN_CAR'])

le.fit(X['FLAG_OWN_REALTY'])
X['FLAG_OWN_REALTY'] = le.transform(X['FLAG_OWN_REALTY'])

le.fit(X['NAME_TYPE_SUITE'].astype(str))
X['NAME_TYPE_SUITE'] = le.transform(X['NAME_TYPE_SUITE'].astype(str))

le.fit(X['NAME_INCOME_TYPE'])
X['NAME_INCOME_TYPE'] = le.transform(X['NAME_INCOME_TYPE'])

le.fit(X['NAME_EDUCATION_TYPE'])
X['NAME_EDUCATION_TYPE'] = le.transform(X['NAME_EDUCATION_TYPE'])

le.fit(X['NAME_FAMILY_STATUS'])
X['NAME_FAMILY_STATUS'] = le.transform(X['NAME_FAMILY_STATUS'])

le.fit(X['NAME_HOUSING_TYPE'].astype(str))
X['NAME_HOUSING_TYPE'] = le.transform(X['NAME_HOUSING_TYPE'].astype(str))

le.fit(X['OCCUPATION_TYPE'].astype(str))
X['OCCUPATION_TYPE'] = le.transform(X['OCCUPATION_TYPE'].astype(str))
                                      
le.fit(X['WEEKDAY_APPR_PROCESS_START'])
X['WEEKDAY_APPR_PROCESS_START'] = le.transform(X['WEEKDAY_APPR_PROCESS_START'])
                                                                         
le.fit(X['ORGANIZATION_TYPE'])
X['ORGANIZATION_TYPE'] = le.transform(X['ORGANIZATION_TYPE'])

X


,DAYS_BIRTH,DAYS_REGISTRATION,DAYS_ID_PUBLISH,DAYS_EMPLOYED,DAYS_LAST_PHONE_CHANGE,AMT_ANNUITY,AMT_INCOME_TOTAL,REGION_POPULATION_RELATIVE,AMT_CREDIT,HOUR_APPR_PROCESS_START,...,NAME_CONTRACT_TYPE,REG_REGION_NOT_WORK_REGION,REG_REGION_NOT_LIVE_REGION,FLAG_DOCUMENT_18,FLAG_DOCUMENT_16,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,DAYS_CREDIT_UPDATE
0,-9461,-3648.0,-2120,-637,-1134.0,24700.5,202500.0,0.018801,406597.5,10,...,0,0,0,0,0,-497,0,-153.0,-153.0,-131
1,-16765,-1186.0,-291,-1188,-828.0,35698.5,270000.0,0.003541,1293502.5,11,...,0,0,0,0,0,-208,0,1075.0,NaN,-20
2,-19046,-4260.0,-2531,-225,-815.0,6750.0,67500.0,0.010032,135000.0,9,...,1,0,0,0,0,-203,0,528.0,NaN,-16
3,-19005,-9833.0,-2437,-3039,-617.0,29686.5,135000.0,0.008019,312682.5,17,...,0,0,0,0,0,-203,0,NaN,NaN,-16
4,-19932,-4311.0,-3458,-3038,-1106.0,21865.5,121500.0,0.028663,513000.0,11,...,0,0,0,0,0,-629,0,1197.0,NaN,-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,-9327,-8456.0,-1982,-236,-273.0,27558.0,157500.0,0.032561,254700.0,15,...,0,0,0,0,0,-147,0,1679.0,NaN,-49
307507,-20775,-4388.0,-4090,365243,0.0,12001.5,72000.0,0.025164,269550.0,8,...,0,0,0,0,0,-1378,0,-1227.0,-1226.0,-1082
307508,-14966,-6737.0,-5150,-7921,-1909.0,29979.0,153000.0,0.005002,677664.0,9,...,0,0,0,0,0,-2178,0,-1966.0,-1966.0,-1966
307509,-11961,-2562.0,-931,-4786,-322.0,20205.0,171000.0,0.005313,370107.0,9,...,0,0,0,0,0,-1189,0,638.0,-833.0,-833


In [4]:
#Splitting Markov feature Data 

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=500)

#Replacing NaNs with means of their column

X_test.fillna(X_train.mean(), inplace=True)
X_test = X_test.fillna(X_train.mean())

X_train.fillna(X_train.mean(), inplace=True)
X_train = X_train.fillna(X_train.mean())


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [5]:
#Fitting HMM 

model = hmm.GMMHMM(n_components=2, n_iter=100, tol = 10e-6, min_covar = 10e-6, random_state = 500)
model.fit(X_train)
model.monitor_.converged

True

In [6]:
#HMM prediction 

y_pred = model.predict(X_test)

Degenerate mixture covariance


In [7]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8611046072998777


In [8]:
# Creating X samples from the HMM model. WIll be used for the GBT model as its input

[X_new, state_seq] = model.sample(n_samples = 307511,random_state = 500)

Degenerate mixture covariance
C:\Users\aguti\AppData\Roaming\Python\Python37\site-packages\hmmlearn\hmm.py:868: RuntimeWarning: covariance is not positive-semidefinite.
  self.means_[state, i_gauss], covs[state]


In [9]:
# Now splitting X data from HMM to train GBT

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_new, Y, test_size=0.20, random_state=500)


In [10]:
# Determining appropriate learning rate

lr_list = [0.01, 0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=10, learning_rate=learning_rate, random_state=500)
    gb_clf.fit(X_train2, y_train2)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training):",(gb_clf.score(X_train2, y_train2)))
    print("Accuracy score (validation):",(gb_clf.score(X_test2, y_test2)))

Learning rate:  0.01
Accuracy score (training): 0.9189782446099314
Accuracy score (validation): 0.9204429052241354
Learning rate:  0.05
Accuracy score (training): 0.9189782446099314
Accuracy score (validation): 0.9204429052241354
Learning rate:  0.075
Accuracy score (training): 0.9189782446099314
Accuracy score (validation): 0.9204429052241354
Learning rate:  0.1
Accuracy score (training): 0.9189782446099314
Accuracy score (validation): 0.9204429052241354
Learning rate:  0.25
Accuracy score (training): 0.919173360215928
Accuracy score (validation): 0.9202152740516722
Learning rate:  0.5
Accuracy score (training): 0.9191855549413027
Accuracy score (validation): 0.9201502365738257
Learning rate:  0.75
Accuracy score (training): 0.9192180742089688
Accuracy score (validation): 0.9201177178349024
Learning rate:  1
Accuracy score (training): 0.9191774251243862
Accuracy score (validation): 0.9201177178349024


In [11]:
gb_clf2 = GradientBoostingClassifier(n_estimators=500, learning_rate=0.01, random_state=500)
gb_clf2.fit(X_train2,y_train2)
predictions = gb_clf2.predict(X_test2)

print("Accuracy:",gb_clf2.score(X_test2,y_test2) )

Accuracy: 0.9203941271157504
